In [407]:
import pandas as pd
import plotly.express as px
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
pd.options.plotting.backend = "plotly"

# Read Data and preprocessing

## 機構評等

In [408]:
df_rating = pd.read_csv("./data/rating.csv",index_col=0)[['日期','股票代號','股票名稱','券商代號','券商名稱','投資評等','目標價']].drop_duplicates()

/var/folders/3w/zdp2qsgd7cz_zklh8f02jsh40000gn/T/ipykernel_39445/514716197.py:1: DtypeWarning:

Columns (17,18) have mixed types. Specify dtype option on import or set low_memory=False.



In [409]:
df_rating['日期'] = pd.to_datetime(df_rating['日期'],format='%Y/%m/%d')
df_rating['股票代號'] = df_rating['股票代號'].astype(pd.StringDtype())
df_rating = df_rating.convert_dtypes().sort_values(by=['日期']).reset_index(drop=True)
df_rating.head(20)

,日期,股票代號,股票名稱,券商代號,券商名稱,投資評等,目標價
0,2008-01-23,2002,中鋼,C0073,富邦證券,增加持股,49.0
1,2008-02-12,2303,聯電,C0086,群益證券,中立,17.9
2,2008-02-12,2454,聯發科,C0073,富邦證券,增加持股,484.0
3,2008-02-12,3037,欣興,C0086,群益證券,買進,50.6
4,2008-02-12,3037,欣興,C0020,元京證券,持有,48.0
5,2008-02-12,2454,聯發科,C0086,群益證券,中立,357.0
6,2008-02-13,2330,台積電,C0070,麥格理證券,Outperform,68.0
7,2008-02-20,2330,台積電,C0020,元京證券,買進,65.0
8,2008-02-20,2454,聯發科,C0020,元京證券,持有,384.0
9,2008-02-20,2382,廣達,C0073,富邦證券,中立,37.0


## 日資料

In [410]:
df_Daily = pd.read_csv("./data/Daily.csv")
df_Daily['日期'] = pd.to_datetime(df_Daily['日期'],format='%Y/%m/%d')
df_Daily["股票代號"] = df_Daily["股票代號"].astype(pd.StringDtype())
df_Daily['本益比'] = df_Daily['本益比'].replace("- -",pd.NA).astype(pd.Float64Dtype())
df_Daily = df_Daily.convert_dtypes()
df_Daily.sort_values(by=['日期'],inplace=True)
df_Daily.reset_index(drop=True,inplace=True)

In [411]:
df_Daily

,日期,股票代號,股票名稱,總市值(億),最高價,最低價,收盤價,本益比,股價淨值比,週轉率(%)
0,2008-01-02,9910,豐泰,126.0,7.96,7.8,7.8,10.3,1.67,0.08
1,2008-01-02,2408,南科,844.8,367.17,353.43,353.43,<NA>,1.29,0.32
2,2008-01-02,2395,研華,349.0,28.04,27.31,27.31,11.4,2.49,0.3
3,2008-01-02,2382,廣達,1565.6,17.27,16.82,16.84,8.4,1.79,0.23
4,2008-01-02,2379,瑞昱,518.1,53.6,50.61,52.68,11.4,3.14,2.77
...,...,...,...,...,...,...,...,...,...,...
176367,2022-12-30,2883,開發金,2321.9,12.7,12.6,12.6,8.9,1.11,0.06
176368,2022-12-30,2882,國泰金,6481.0,40.25,39.6,40.0,9.7,1.87,0.63
176369,2022-12-30,2881,富邦金,7879.3,56.8,56.2,56.3,8.7,1.43,0.04
176370,2022-12-30,3008,大立光,2723.4,2043.57,1999.46,1999.46,11.6,1.8,0.24


## 外資持股比例

In [412]:
df_holding = pd.read_csv("./data/外資持股.csv",index_col=0)
df_holding['日期'] = pd.to_datetime(df_holding['日期'],format='%Y/%m/%d')
df_holding['股票代號'] = df_holding['股票代號'].astype(pd.StringDtype())
df_holding = df_holding.convert_dtypes().sort_values(by='日期',ascending=True)
df_holding['外資持股比率(%)'] = (df_holding['外資持股比率(%)']/100)
df_holding.rename(columns={'外資持股比率(%)':'外資持股比率'},inplace=True)
df_holding.reset_index(drop=True,inplace=True)

In [413]:
df_holding.head(20)

,日期,股票代號,股票名稱,外資持股比率
0,2008-01-02,9910,豐泰,0.0625
1,2008-01-02,2408,南科,0.0661
2,2008-01-02,2395,研華,0.3359
3,2008-01-02,2382,廣達,0.1981
4,2008-01-02,2379,瑞昱,0.2076
5,2008-01-02,2357,華碩,0.3789
6,2008-01-02,2330,台積電,0.7179
7,2008-01-02,2327,國巨,0.4295
8,2008-01-02,2317,鴻海,0.5649
9,2008-01-02,2308,台達電,0.7382


# 計算預測誤差

## 預測誤差敘述性統計
目標價在不同預測期間的誤差

In [414]:
periods = list(map(lambda D: pd.Timedelta(D,"d"),[30,60,90,180,360]))

In [415]:
df_rating.head(10)

,日期,股票代號,股票名稱,券商代號,券商名稱,投資評等,目標價
0,2008-01-23,2002,中鋼,C0073,富邦證券,增加持股,49.0
1,2008-02-12,2303,聯電,C0086,群益證券,中立,17.9
2,2008-02-12,2454,聯發科,C0073,富邦證券,增加持股,484.0
3,2008-02-12,3037,欣興,C0086,群益證券,買進,50.6
4,2008-02-12,3037,欣興,C0020,元京證券,持有,48.0
5,2008-02-12,2454,聯發科,C0086,群益證券,中立,357.0
6,2008-02-13,2330,台積電,C0070,麥格理證券,Outperform,68.0
7,2008-02-20,2330,台積電,C0020,元京證券,買進,65.0
8,2008-02-20,2454,聯發科,C0020,元京證券,持有,384.0
9,2008-02-20,2382,廣達,C0073,富邦證券,中立,37.0


In [416]:
df_Daily.head(10)

,日期,股票代號,股票名稱,總市值(億),最高價,最低價,收盤價,本益比,股價淨值比,週轉率(%)
0,2008-01-02,9910,豐泰,126.0,7.96,7.8,7.8,10.3,1.67,0.08
1,2008-01-02,2408,南科,844.8,367.17,353.43,353.43,<NA>,1.29,0.32
2,2008-01-02,2395,研華,349.0,28.04,27.31,27.31,11.4,2.49,0.3
3,2008-01-02,2382,廣達,1565.6,17.27,16.82,16.84,8.4,1.79,0.23
4,2008-01-02,2379,瑞昱,518.1,53.6,50.61,52.68,11.4,3.14,2.77
5,2008-01-02,2357,華碩,3616.5,129.17,125.4,126.05,10.5,2.25,0.41
6,2008-01-02,2330,台積電,15988.4,35.55,34.52,34.69,13.7,3.53,0.15
7,2008-01-02,2327,國巨,268.2,27.42,26.35,26.35,4.8,0.84,0.46
8,2008-01-02,2317,鴻海,12235.6,61.45,59.17,59.17,12.3,3.49,0.19
9,2008-01-02,2308,台達電,2243.3,60.5,58.58,58.58,12.0,3.75,0.16


## 合併機構預測與當日股價資料集

In [417]:
df_rating_daily = df_rating.merge(df_Daily,how='left',on=['日期','股票代號','股票名稱'])

In [418]:
df_tp = df_rating_daily[['日期','券商代號','券商名稱','目標價','收盤價','股票代號','股票名稱']].copy()

In [419]:
df_tp['多/空'] = df_tp['目標價'] > df_tp['收盤價']

In [420]:
df_tp.dropna(inplace=True)
df_tp

,日期,券商代號,券商名稱,目標價,收盤價,股票代號,股票名稱,多/空
0,2008-01-23,C0073,富邦證券,49.0,18.67,2002,中鋼,True
1,2008-02-12,C0086,群益證券,17.9,8.75,2303,聯電,True
2,2008-02-12,C0073,富邦證券,484.0,138.86,2454,聯發科,True
3,2008-02-12,C0086,群益證券,50.6,24.93,3037,欣興,True
4,2008-02-12,C0020,元京證券,48.0,24.93,3037,欣興,True
...,...,...,...,...,...,...,...,...
29090,2022-12-29,C0019,元大投顧,185.0,121.5,3037,欣興,True
29091,2022-12-29,C0019,元大投顧,353.0,281.0,2308,台達電,True
29092,2022-12-30,C0049,花旗美邦證券,550.0,446.09,2330,台積電,True
29093,2022-12-30,C0006,大和,320.0,286.5,2308,台達電,True


## 未來一段時間股價極值

In [421]:
df_daily_min_max = df_Daily[['日期','股票代號','最高價','最低價']].pivot(index='日期',columns='股票代號',values=['最高價','最低價'])
df_daily_max = df_daily_min_max.groupby(axis=1,level=0).get_group('最高價').resample('1D').first().fillna(method='ffill')
rolling_maxs = [df_daily_max.rolling(period.days).max().shift(-period.days).stack().reset_index() for period in periods]
df_daily_min = df_daily_min_max.groupby(axis=1,level=0).get_group('最低價').resample('1D').first().fillna(method='ffill')
rolling_mins = [df_daily_min.rolling(period.days).min().shift(-period.days).stack().reset_index() for period in periods]

In [422]:
rolling_maxs[0]

,日期,股票代號,最高價
0,2008-01-02,1101,19.30
1,2008-01-02,1216,18.30
2,2008-01-02,1301,45.05
3,2008-01-02,1303,42.43
4,2008-01-02,1326,40.07
...,...,...,...
259942,2022-11-30,5876,49.80
259943,2022-11-30,5880,26.75
259944,2022-11-30,6415,573.92
259945,2022-11-30,6505,85.50


## 股價動能

In [423]:
period = pd.Timedelta(360,"d")

In [424]:
# 30day return for momentum
momentum = df_Daily[['日期','股票代號','收盤價']].pivot(index='日期',columns='股票代號',values=['收盤價'])
momentum = (momentum.shift(period.days)/momentum - 1)
momentum = momentum.stack().reset_index()[['日期','股票代號','收盤價']]
momentum.rename(columns={'收盤價':f'{period.days}日報酬率'},inplace=True)
momentum.dropna(inplace=True)

In [425]:
momentum

,日期,股票代號,360日報酬率
0,2009-06-18,1101,0.439183
1,2009-06-18,1216,0.157475
2,2009-06-18,1301,0.341115
3,2009-06-18,1303,0.739955
4,2009-06-18,1326,0.476721
...,...,...,...
158218,2022-12-30,5876,-0.059251
158219,2022-12-30,5880,-0.268077
158220,2022-12-30,6415,1.221716
158221,2022-12-30,6505,0.188294


## 合併股價動能到資料集

In [426]:
df_rating_daily_momentum = df_rating_daily.merge(momentum,how='left',on=['日期','股票代號']).dropna()

In [427]:
df_rating_daily_momentum.head(10)

,日期,股票代號,股票名稱,券商代號,券商名稱,投資評等,目標價,總市值(億),最高價,最低價,收盤價,本益比,股價淨值比,週轉率(%),360日報酬率
1353,2009-06-19,2454,聯發科,C0089,德意志證券,Sell,223.0,4067.4,207.12,198.94,206.57,15.5,4.55,0.81,-0.048361
1354,2009-06-19,3034,聯詠,C0073,富邦證券,增加持股,85.0,433.3,30.56,29.2,30.56,15.9,2.29,1.88,0.385144
1355,2009-06-22,1402,遠紡,C0060,高盛證券,Buy,41.1,1748.0,18.19,17.12,18.19,24.2,2.0,0.97,-0.047828
1356,2009-06-22,2382,廣達,C0048,法銀巴黎,Hold,53.0,1928.1,22.11,21.41,22.11,10.4,2.1,0.41,-0.263229
1357,2009-06-22,2379,瑞昱,C0040,里昂證券,Outperform,62.0,276.5,29.05,28.17,28.85,20.0,1.75,2.81,0.658579
1358,2009-06-22,2887,台新金,C0034,永豐金,買進,14.5,807.1,4.35,4.18,4.26,25.7,0.95,1.07,0.152582
1360,2009-06-22,3034,聯詠,C0070,麥格理證券,Outperform,90.0,436.3,31.02,29.99,30.77,16.1,2.31,1.51,0.346766
1361,2009-06-23,2454,聯發科,C0086,群益證券,買進,438.0,4024.5,206.02,204.39,204.39,15.3,4.5,0.45,-0.157836
1362,2009-06-24,2912,統一超,C0086,群益證券,中立,79.0,776.1,47.8,46.14,47.14,20.6,4.48,0.32,-0.056216
1363,2009-06-24,2454,聯發科,C0089,德意志證券,Sell,223.0,4147.9,215.84,205.48,210.66,15.8,4.64,0.87,-0.225957


## 計算誤差

In [428]:
def calc_error(x):
    # if(x['日期'] == pd.to_datetime("2020-10-07")):
    #     print(x)
    return x['目標價']/x['最高價'] - 1 if x['多/空'] else 1-x['目標價']/x['最低價']

In [429]:
tp_errors = {}
for i,p in enumerate(periods):
    df_tmp = df_tp.copy()
    df_tmp['日期'] = df_tmp['日期'] + p
    df_tmp = df_tmp.merge(rolling_maxs[i],how='left',on=['日期','股票代號']).merge(rolling_mins[i],how='left',on=['日期','股票代號'])
    tp_errors[f'{p.days}天後誤差'] = np.array(df_tmp.apply(func=calc_error,axis=1))
for k,v in tp_errors.items():
    df_tp[k] = v

In [430]:
df_tp.dropna(inplace=True)

In [431]:
df_tp.head(10)

,日期,券商代號,券商名稱,目標價,收盤價,股票代號,股票名稱,多/空,30天後誤差,60天後誤差,90天後誤差,180天後誤差,360天後誤差
0,2008-01-23,C0073,富邦證券,49.0,18.67,2002,中鋼,True,1.259106,1.001634,1.001634,1.163355,1.520576
1,2008-02-12,C0086,群益證券,17.9,8.75,2303,聯電,True,0.768775,0.708015,0.708015,1.277354,0.645221
2,2008-02-12,C0073,富邦證券,484.0,138.86,2454,聯發科,True,1.261999,1.160232,1.254098,1.336810,0.455114
3,2008-02-12,C0086,群益證券,50.6,24.93,3037,欣興,True,0.740028,0.698557,0.713512,1.014331,0.542213
4,2008-02-12,C0020,元京證券,48.0,24.93,3037,欣興,True,0.650619,0.611279,0.625466,0.910828,0.462969
5,2008-02-12,C0086,群益證券,357.0,138.86,2454,聯發科,True,0.668458,0.593394,0.662630,0.723638,0.073297
6,2008-02-13,C0070,麥格理證券,68.0,34.35,2330,台積電,True,0.711122,0.699150,0.699150,0.797515,0.614435
7,2008-02-20,C0020,元京證券,65.0,35.03,2330,台積電,True,0.635632,0.624188,0.645153,0.749193,0.543210
8,2008-02-20,C0020,元京證券,384.0,165.83,2454,聯發科,True,0.713903,0.788376,0.842434,0.863897,0.154471
9,2008-02-20,C0073,富邦證券,37.0,14.65,2382,廣達,True,1.016349,0.864919,0.801363,0.794374,0.114794


In [432]:
df_tp[[f'{p.days}天後誤差' for p in periods]].abs().describe()

,30天後誤差,60天後誤差,90天後誤差,180天後誤差,360天後誤差
count,22406.000000,22406.000000,22406.000000,22406.000000,22406.000000
mean,0.592775,0.555903,0.534757,0.474918,0.427067
std,0.547967,0.543551,0.539159,0.464238,0.396835
min,0.000000,0.000000,0.000000,0.000000,0.000054
25%,0.233266,0.202259,0.188912,0.167437,0.147738
50%,0.448749,0.409605,0.387847,0.341629,0.317637
75%,0.780659,0.731333,0.702976,0.632221,0.584607
max,12.164310,11.646370,11.646370,11.765957,11.844910


In [433]:
df_tp[[f'{p.days}天後誤差' for p in periods]].hist()

In [434]:
df_tp[df_tp['360天後誤差'].abs() >= df_tp['360天後誤差'].abs().max()]

,日期,券商代號,券商名稱,目標價,收盤價,股票代號,股票名稱,多/空,30天後誤差,60天後誤差,90天後誤差,180天後誤差,360天後誤差
18227,2018-11-05,C0067,統一證券,270.0,20.39,2002,中鋼,True,12.16431,11.64637,11.64637,11.765957,11.84491


## 結論

券商的目標價在預測360天後的股價有較高的準確率 \
故此研究以360天作為計算預測誤差的期間

# 驗證假說

## 假說一：外資券商對於目標價預測誤差優於本土券商

In [435]:
# 是否為本土券商
domestic = {'富邦證券','群益證券','元京證券','凱基證券','元富證券投顧','統一證券','日盛證券','元大投顧','永豐金','中國信託綜合證券','群益證券','兆豐','國票','國泰','第一金證券','台新投顧',
            '元京證券','宏遠證券','大華證券','康和證券','玉山證券','台工銀','大和國泰證券','奔亞投顧','金鼎證券','合作金庫證券','華南永昌','凱基'}

In [436]:
in_domestic = df_tp['券商名稱'].apply(lambda x: float(x in domestic)).rename('是否為本土券商')

In [437]:
df_implied_ret = (df_rating_daily_momentum['目標價'] / df_rating_daily_momentum['收盤價'] - 1).rename("隱含報酬率")

In [438]:
df_reg1 = pd.concat([df_rating_daily_momentum,in_domestic,df_tp['360天後誤差'],df_implied_ret]
                    ,axis=1)[['360天後誤差','360日報酬率'
                              ,'總市值(億)','本益比','股價淨值比',
                              '是否為本土券商','週轉率(%)',"隱含報酬率"]].dropna()
df_reg1

,360天後誤差,360日報酬率,總市值(億),本益比,股價淨值比,是否為本土券商,週轉率(%),隱含報酬率
1353,-0.250722,-0.048361,4067.4,15.5,4.55,0.0,0.81,0.079537
1354,0.774160,0.385144,433.3,15.9,2.29,1.0,1.88,1.781414
1355,0.503842,-0.047828,1748.0,24.2,2.0,0.0,0.97,1.259483
1356,0.624770,-0.263229,1928.1,10.4,2.1,0.0,0.41,1.397105
1357,0.514042,0.658579,276.5,20.0,1.75,0.0,2.81,1.149047
...,...,...,...,...,...,...,...,...
23041,0.347639,0.338445,4076.6,14.8,2.9,1.0,1.45,0.241637
23042,0.109821,0.338445,4076.6,14.8,2.9,0.0,1.45,0.022525
23043,0.228730,0.338445,4076.6,14.8,2.9,1.0,1.45,0.132081
23044,0.397185,0.338445,4076.6,14.8,2.9,1.0,1.45,0.287286


In [439]:
df_reg1.columns = ['tp_error','momentum','Size','PE','PB','Domestic','turnover','implied_ret']
df_reg1.head(10)

,tp_error,momentum,Size,PE,PB,Domestic,turnover,implied_ret
1353,-0.250722,-0.048361,4067.4,15.5,4.55,0.0,0.81,0.079537
1354,0.774160,0.385144,433.3,15.9,2.29,1.0,1.88,1.781414
1355,0.503842,-0.047828,1748.0,24.2,2.0,0.0,0.97,1.259483
1356,0.624770,-0.263229,1928.1,10.4,2.1,0.0,0.41,1.397105
1357,0.514042,0.658579,276.5,20.0,1.75,0.0,2.81,1.149047
1358,1.077364,0.152582,807.1,25.7,0.95,1.0,1.07,2.403756
1360,0.878522,0.346766,436.3,16.1,2.31,0.0,1.51,1.924927
1361,0.471675,-0.157836,4024.5,15.3,4.5,1.0,0.45,1.142962
1362,-0.311967,-0.056216,776.1,20.6,4.48,1.0,0.32,0.675859
1363,-0.250722,-0.225957,4147.9,15.8,4.64,0.0,0.87,0.058578


In [440]:
model1 = smf.ols("np.abs(tp_error) ~ np.log(Size) + PE + PB + Domestic + turnover + momentum + implied_ret",data=df_reg1.astype(np.float64))

In [441]:
results1 = model1.fit(cov_type="HC0")
results1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       np.abs(tp_error)   R-squared:                       0.327
Model:                            OLS   Adj. R-squared:                  0.327
Method:                 Least Squares   F-statistic:                     302.7
Date:                Wed, 24 May 2023   Prob (F-statistic):               0.00
Time:                        22:12:53   Log-Likelihood:                -5682.9
No. Observations:               20750   AIC:                         1.138e+04
Df Residuals:                   20742   BIC:                         1.145e+04
Df Model:                           7                                         
Covariance Type:                  HC0                                         
================================================================================
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        0.3003      0.030      9.888      0.000       0.241       0.360
np.log(Size)    -0.0159      0.003     -6.041      0.000      -0.021      -0.011
PE               0.0001   8.16e-05      1.689      0.091   -2.21e-05       0.000
PB              -0.0136      0.002     -7.021      0.000      -0.017      -0.010
Domestic         0.0123      0.004      2.781      0.005       0.004       0.021
turnover         0.0416      0.003     12.327      0.000       0.035       0.048
momentum        -0.1576      0.010    -15.214      0.000      -0.178      -0.137
implied_ret      0.3595      0.016     22.338      0.000       0.328       0.391
==============================================================================
Omnibus:                     8782.288   Durbin-Watson:                   1.315
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           268160.069
Skew:                           1.420   Prob(JB):                         0.00
Kurtosis:                      20.381   Cond. No.                         312.
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC0)
"""

In [442]:
results1.t_test("Domestic = 0")

<class 'statsmodels.stats.contrast.ContrastResults'>
                             Test for Constraints                             
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
c0             0.0123      0.004      2.781      0.005       0.004       0.021

In [443]:
df_reg1[df_reg1.columns[1:]].corr()

,momentum,Size,PE,PB,Domestic,turnover,implied_ret
momentum,1.000000,-0.118995,0.019284,-0.343778,-0.013778,-0.150916,-0.067183
Size,-0.118995,1.000000,-0.009099,0.148525,-0.082443,-0.138624,-0.146102
PE,0.019284,-0.009099,1.000000,0.088908,-0.006343,-0.002280,-0.062181
PB,-0.343778,0.148525,0.088908,1.000000,-0.094100,0.069269,-0.100234
Domestic,-0.013778,-0.082443,-0.006343,-0.094100,1.000000,0.067788,0.051721
turnover,-0.150916,-0.138624,-0.002280,0.069269,0.067788,1.000000,-0.068653
implied_ret,-0.067183,-0.146102,-0.062181,-0.100234,0.051721,-0.068653,1.000000


## 假說二：台灣五十成分股中的公司，券商預測能力越準確（預測誤差愈小）

這裡選用:
- 玉山金 (臺證所：2884)
- 臺新金 (臺證所：2887)
- 研華（臺證所：2395）
- 上海商銀(臺證所：5876)
- 和泰汽車(臺證所：2207)

In [444]:
add_in_50 = {"2884","4938","2395","5876","2207"}

In [445]:
(df_rating_daily_momentum["股票代號"] == "2379").value_counts()

股票代號
False    25736
True       816
Name: count, dtype: Int64

## 假說三：外資持股比例高的公司，外資券商偏向高估股價（高估的預測誤差愈大）

In [446]:
df_reg3 = df_rating_daily_momentum.merge(df_holding,how='left',on=['日期','股票代號','股票名稱']).dropna()
df_reg3 = pd.concat([df_reg3,in_domestic,df_tp['360天後誤差'],df_implied_ret],
                    axis=1,join='inner')[['360天後誤差','360日報酬率'
                              ,'總市值(億)','本益比','股價淨值比',
                              '是否為本土券商','週轉率(%)',
                              "隱含報酬率","外資持股比率"]].dropna()
df_reg3.columns = ['tp_error','momentum','Size','PE','PB','Is_foriegn','turnover','implied_ret',"foreign_holding"]

In [447]:
df_reg3['Is_foriegn'] = 1 - df_reg3['Is_foriegn'] 

In [448]:
model3 = smf.ols("tp_error ~ np.log(Size) + PE + PB + Is_foriegn + turnover + momentum + implied_ret + foreign_holding + Is_foriegn*foreign_holding",data=df_reg3.astype(np.float64))

In [449]:
result3 = model3.fit(cov_type="HC0")
result3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               tp_error   R-squared:                       0.408
Model:                            OLS   Adj. R-squared:                  0.408
Method:                 Least Squares   F-statistic:                     348.0
Date:                Wed, 24 May 2023   Prob (F-statistic):               0.00
Time:                        22:12:54   Log-Likelihood:                -11132.
No. Observations:               20750   AIC:                         2.228e+04
Df Residuals:                   20740   BIC:                         2.236e+04
Df Model:                           9                                         
Covariance Type:                  HC0                                         
==============================================================================================
                                 coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept                      0.0666      0.026      2.526      0.012       0.015       0.118
np.log(Size)                  -0.0233      0.003     -8.989      0.000      -0.028      -0.018
PE                            -0.0001   8.35e-05     -1.509      0.131      -0.000    3.76e-05
PB                             0.0007      0.001      0.490      0.624      -0.002       0.003
Is_foriegn                    -0.0190      0.015     -1.296      0.195      -0.048       0.010
turnover                      -0.0212      0.003     -6.780      0.000      -0.027      -0.015
momentum                       0.1601      0.012     13.480      0.000       0.137       0.183
implied_ret                    0.5780      0.014     40.042      0.000       0.550       0.606
foreign_holding               -0.0130      0.022     -0.603      0.546      -0.055       0.029
Is_foriegn:foreign_holding     0.0032      0.030      0.104      0.917      -0.056       0.062
==============================================================================
Omnibus:                     1952.279   Durbin-Watson:                   1.257
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            14723.709
Skew:                           0.013   Prob(JB):                         0.00
Kurtosis:                       7.127   Cond. No.                         474.
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC0)
"""